# CNN for Cifar-10 Dataset

## Requirements

For this project we are used:
* Python 3.6
* Tensorflow 1.8.0
* *GPUs parallel calculation manager* nVidia CUDA 9.0
* *GPU-accelerated library* nVidia cuDNN 7.1
* or CPU optimized tensorflow for intel

## Introduction

In this first notebook we are going to explain how we have set up our work.  
Essentially the process is divided into several parts:
* Set up environment with the Cifar-10 Dataset
* Define a convolutional neural network
* Define a quantization method
* Train the convolutional neural network
* Provide information about CNN's performance and accuracy

In [5]:
import numpy as np
import tensorflow as tf

## Cifar-10 Dataset

Cifar-10 Dataset is taken from the official website www.cs.toronto.edu.

Dataset is stored in the data directory: cnn/data. From Cifar-10 dataset we are going to take x_train, t_train, x_test and t_test.
The training dataset set is used for training the CNN, the testing dataset is used for evaluate the performance and the accuracy of the network.

### Load data

In [7]:
from cnn.utils.dataset import load_cifar10
from cnn.dense import dataset_preprocessing_by_keras

In [8]:
x_train, t_train, x_test, t_test = load_cifar10()

In [9]:
x_train.shape, t_train.shape, x_test.shape, t_test.shape

((50000, 32, 32, 3), (50000, 10), (10000, 32, 32, 3), (10000, 10))

In [13]:
x_train = dataset_preprocessing_by_keras(x_train)
x_train[0, :, :, 0]

array([[-0.68748678, -0.65573727, -0.63986251, ..., -0.95735764,
        -1.00498191, -0.92560813],
       [-0.65573727, -0.63986251, -0.59223824, ..., -0.92560813,
        -1.02085667, -0.8621091 ],
       [-0.71923629, -0.63986251, -0.65573727, ..., -0.68748678,
        -0.73511105, -0.70336154],
       ...,
       [-0.36999165, -0.75098581, -1.00498191, ..., -0.25886835,
        -0.24299359, -0.14774506],
       [-0.25886835, -0.65573727, -1.19547899, ...,  0.31262289,
        -0.00487225, -0.4969897 ],
       [-0.27474311, -0.35411689, -1.11610521, ...,  0.75711607,
         0.64599278,  0.28087337]])

## CNN Model and Training

We will use a custom made wrapper for tensorfllow NN training and use

In [2]:
from cnn.model_class import TfClassifier

This CNN is called *dense_cnn*. Here we will explain how it is composed.

The CNN is composed by several layers. In the first part there are 2 **convolutional** layers and 2 **pooling** layers (they are alternated), then there are a *flatten* layer followed by a **relu** layer, a *dropout* layer and finally a **softmax** layer.

The network uses a stochastic gradient descent optimizer and a categorical crossentropy loss.  
To judge the performance of our model we are used a MSE metric.

In [14]:
from cnn.dense import NET_NAME, forward_pass, loss_fn, eval_fn

In [15]:
model = TfClassifier(NET_NAME, forward_pass, loss_fn, eval_fn,
                     tf.train.AdamOptimizer())

This network is trained for 50 epochs.

In [17]:
history = model.fit(
    [x_train, t_train], batch_size=64, validation_split=0.1, epochs=1, verbosity=1)

print(history)

For training: tensorboard --logdir=/tmp/log-tb/dense_cnn/training
For validation: tensorboard --logdir=/tmp/log-tb/dense_cnn/validation
{'logits': (array([[-0.5063772 , -4.981158  ,  0.43995675, ..., -0.38799834,
        -2.73253   , -5.3762517 ],
       [ 3.451639  ,  0.8174111 , -0.80418146, ..., -2.1745582 ,
         4.976637  ,  0.8644375 ],
       [-1.4941906 ,  5.4328566 , -0.25882745, ..., -3.0065315 ,
        -0.9388854 ,  1.785409  ],
       ...,
       [-1.5114052 , -3.1864355 , -0.04986552, ...,  2.3482933 ,
        -3.1718757 , -1.6047599 ],
       [-0.6918783 , -1.880009  ,  0.30249122, ...,  2.175767  ,
        -2.4612677 , -1.2348982 ],
       [-0.37802106, -2.5042136 ,  0.7668707 , ..., -2.3310952 ,
         5.109047  , -1.0824846 ]], dtype=float32),), 'classes': (array([3, 8, 1, ..., 7, 4, 8]),), 'probabilities': (array([[8.05834495e-03, 9.18064325e-05, 2.07602903e-02, ...,
        9.07103904e-03, 8.69841082e-04, 6.18423583e-05],
       [1.73318610e-01, 1.24398321e-02,

Then it's evaluated

In [ ]:
evals = model.evaluate([x_test, t_test])

print(evals)